# Tensors

## Introduction

In [1]:
import torch
import math

In [2]:
a = [1.0, 2.0, 1.0]

In [3]:
a[0]

1.0

In [4]:
a[2] = 3.0
a

[1.0, 2.0, 3.0]

In [5]:
a = torch.ones(3)
a

tensor([1., 1., 1.])

In [6]:
a[1]

tensor(1.)

In [7]:
float(a[1])

1.0

In [8]:
a[2] = 2.0
a

tensor([1., 1., 2.])

In [9]:
points = torch.zeros(6)
points[0] = 4.0
points[1] = 1.0
points[2] = 5.0
points[3] = 3.0
points[4] = 2.0
points[5] = 1.0
points

tensor([4., 1., 5., 3., 2., 1.])

In [10]:
points = torch.tensor([4.0, 1.0, 5.0, 3.0, 2.0, 1.0])
points

tensor([4., 1., 5., 3., 2., 1.])

In [11]:
float(points[0]), float(points[1])

(4.0, 1.0)

In [12]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [13]:
points.shape

torch.Size([3, 2])

In [14]:
points = torch.zeros(3, 2)
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [15]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points[0, 1]

tensor(1.)

In [16]:
points[0]

tensor([4., 1.])

## Indexing Tensors

In [17]:
points[1:]

tensor([[5., 3.],
        [2., 1.]])

In [18]:
points[1:, :]

tensor([[5., 3.],
        [2., 1.]])

In [19]:
points[1:, 0]

tensor([5., 2.])

In [20]:
points[None]

tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])

## Named Tensors

In [21]:
img_t = torch.randn(3, 5, 5)
weights = torch.tensor([0.2126, 0.7152, 0.0722])

In [22]:
batch_t = torch.randn(2, 3, 5, 5)

In [23]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [24]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [25]:
img_gray_weighted_fancy = torch.einsum("...chw,c->...hw", img_t, weights)
batch_gray_weighted_fancy = torch.einsum("...chw,c->...hw", batch_t, weights)
batch_gray_weighted_fancy.shape

torch.Size([2, 5, 5])

In [26]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=["channels"])
weights_named

/tmp/ipykernel_30900/2918111397.py:1: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /opt/conda/conda-bld/pytorch_1708025845206/work/c10/core/TensorImpl.h:1908.)
  weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=["channels"])


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [27]:
img_named = img_t.refine_names(..., "channels", "rows", "columns")
batch_named = batch_t.refine_names(..., "channels", "rows", "columns")
print("img_named:", img_named.shape, img_named.names)
print("batch_named:", batch_named.shape, batch_named.names)

img_named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch_named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [28]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [29]:
gray_named = (img_named * weights_aligned).sum("channels")
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [30]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))

## Tensor Element Types

In [31]:
a = torch.zeros(10, 2)
a.dtype

torch.float32

In [32]:
b = torch.tensor([1, 2])
b.dtype

torch.int64

In [33]:
c = (b > 1.0)
type(c), c.dtype

(torch.Tensor, torch.bool)

In [34]:
double_points = torch.ones(10, 2, dtype=torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype=torch.short)
double_points.dtype, short_points.dtype

(torch.float64, torch.int16)

In [35]:
double_points = torch.zeros(10, 2).double()
short_points = torch.ones(10, 2).short()
double_points.dtype, short_points.dtype

(torch.float64, torch.int16)

In [36]:
double_points = torch.zeros(10, 2).to(torch.double)
short_points = torch.ones(10, 2).to(dtype=torch.short)
double_points.dtype, short_points.dtype

(torch.float64, torch.int16)

In [37]:
points_64 = torch.rand(5, dtype=torch.double)
points_short = points_64.to(torch.short)
points_64 * points_short

tensor([0., 0., 0., 0., 0.], dtype=torch.float64)

## Tensor API

In [38]:
x = torch.empty(3, 4)
print(type(x))
print(x)

<class 'torch.Tensor'>
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])


In [39]:
zeros = torch.zeros(2, 3)
print(zeros)

ones = torch.ones(2, 3)
print(ones)

torch.manual_seed(42)
random = torch.rand(2, 3)
print(random)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([[0.8823, 0.9150, 0.3829],
        [0.9593, 0.3904, 0.6009]])


In [40]:
x = torch.empty(2, 2, 3)
print(x.shape)
print(x)

empty_like_x = torch.empty_like(x)
print(empty_like_x.shape)
print(empty_like_x)

zeros_like_x = torch.zeros_like(x)
print(zeros_like_x.shape)
print(zeros_like_x)

ones_like_x = torch.ones_like(x)
print(ones_like_x.shape)
print(ones_like_x)

rand_like_x = torch.rand_like(x)
print(rand_like_x.shape)
print(rand_like_x)

torch.Size([2, 2, 3])
tensor([[[-3.3579e+36,  4.4226e-41, -1.1876e-28],
         [ 3.1672e-41,  1.4013e-45,  0.0000e+00]],

        [[ 1.4013e-45,  0.0000e+00,  1.4013e-45],
         [ 0.0000e+00,  1.4013e-45,  0.0000e+00]]])
torch.Size([2, 2, 3])
tensor([[[-3.3579e+36,  4.4226e-41, -3.3579e+36],
         [ 4.4226e-41,  1.4013e-45,  0.0000e+00]],

        [[ 1.4013e-45,  0.0000e+00,  1.4013e-45],
         [ 0.0000e+00,  1.4013e-45,  0.0000e+00]]])
torch.Size([2, 2, 3])
tensor([[[0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.]]])
torch.Size([2, 2, 3])
tensor([[[1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.]]])
torch.Size([2, 2, 3])
tensor([[[0.2566, 0.7936, 0.9408],
         [0.1332, 0.9346, 0.5936]],

        [[0.8694, 0.5677, 0.7411],
         [0.4294, 0.8854, 0.5739]]])


In [41]:
some_constants = torch.tensor([[3.1415926, 2.71828], [1.61803, 0.0072897]])
print(some_constants)

some_integers = torch.tensor([2, 3, 5, 7, 11, 13, 17, 19])
print(some_integers)

more_integers = torch.tensor(((2, 4, 6), [3, 6, 9]))
print(more_integers)

tensor([[3.1416, 2.7183],
        [1.6180, 0.0073]])
tensor([ 2,  3,  5,  7, 11, 13, 17, 19])
tensor([[2, 4, 6],
        [3, 6, 9]])


In [42]:
a = torch.ones((2, 3), dtype=torch.int16)
print(a)

b = torch.rand((2, 3), dtype=torch.float64) * 20
print(b)

c = b.to(torch.int32)
print(c)

tensor([[1, 1, 1],
        [1, 1, 1]], dtype=torch.int16)
tensor([[ 2.6528,  3.1411,  7.5073],
        [16.8504, 17.4100,  7.5507]], dtype=torch.float64)
tensor([[ 2,  3,  7],
        [16, 17,  7]], dtype=torch.int32)


In [43]:
ones = torch.zeros(2, 2) + 1
twos = torch.ones(2, 2) * 2
threes = (torch.ones(2, 2) * 7 - 1) / 2
fours = twos ** 2
sqrt2s = twos ** 0.5

print(ones)
print(twos)
print(threes)
print(fours)
print(sqrt2s)

tensor([[1., 1.],
        [1., 1.]])
tensor([[2., 2.],
        [2., 2.]])
tensor([[3., 3.],
        [3., 3.]])
tensor([[4., 4.],
        [4., 4.]])
tensor([[1.4142, 1.4142],
        [1.4142, 1.4142]])


In [44]:
powers2 = twos ** torch.tensor([[1, 2], [3, 4]])
print(powers2)

fives = ones + fours
print(fives)

dozens = threes * fours
print(dozens)

tensor([[ 2.,  4.],
        [ 8., 16.]])
tensor([[5., 5.],
        [5., 5.]])
tensor([[12., 12.],
        [12., 12.]])


In [45]:
rand = torch.rand(2, 4)
doubled = rand * (torch.ones(1, 4) * 2)

print(rand)
print(doubled)

tensor([[0.9516, 0.0753, 0.8860, 0.5832],
        [0.3376, 0.8090, 0.5779, 0.9040]])
tensor([[1.9031, 0.1505, 1.7720, 1.1664],
        [0.6753, 1.6179, 1.1559, 1.8080]])


In [46]:
a = torch.ones(4, 3, 2)

b = a * torch.rand(3, 2)
print(b)

c = a * torch.rand(3, 1)
print(c)

d = a * torch.rand(1, 2)
print(d)

tensor([[[0.5547, 0.3423],
         [0.6343, 0.3644],
         [0.7104, 0.9464]],

        [[0.5547, 0.3423],
         [0.6343, 0.3644],
         [0.7104, 0.9464]],

        [[0.5547, 0.3423],
         [0.6343, 0.3644],
         [0.7104, 0.9464]],

        [[0.5547, 0.3423],
         [0.6343, 0.3644],
         [0.7104, 0.9464]]])
tensor([[[0.7890, 0.7890],
         [0.2814, 0.2814],
         [0.7886, 0.7886]],

        [[0.7890, 0.7890],
         [0.2814, 0.2814],
         [0.7886, 0.7886]],

        [[0.7890, 0.7890],
         [0.2814, 0.2814],
         [0.7886, 0.7886]],

        [[0.7890, 0.7890],
         [0.2814, 0.2814],
         [0.7886, 0.7886]]])
tensor([[[0.5895, 0.7539],
         [0.5895, 0.7539],
         [0.5895, 0.7539]],

        [[0.5895, 0.7539],
         [0.5895, 0.7539],
         [0.5895, 0.7539]],

        [[0.5895, 0.7539],
         [0.5895, 0.7539],
         [0.5895, 0.7539]],

        [[0.5895, 0.7539],
         [0.5895, 0.7539],
         [0.5895, 0.7539]]])


In [47]:
a = torch.rand(2, 4) * 2 - 1
print("Common functions:")
print(f"abs(a)\n{torch.abs(a)}")
print(f"ceil(a)\n{torch.ceil(a)}")
print(f"floor(a)\n{torch.floor(a)}")
print(f"clamp(a, -0.5, 0.5)\n{torch.clamp(a, -0.5, 0.5)}")

Common functions:
abs(a)
tensor([[0.6095, 0.9899, 0.3864, 0.7670],
        [0.8205, 0.2880, 0.4142, 0.3163]])
ceil(a)
tensor([[-0., -0., -0., -0.],
        [1., 1., 1., 1.]])
floor(a)
tensor([[-1., -1., -1., -1.],
        [ 0.,  0.,  0.,  0.]])
clamp(a, -0.5, 0.5)
tensor([[-0.5000, -0.5000, -0.3864, -0.5000],
        [ 0.5000,  0.2880,  0.4142,  0.3163]])


In [48]:
angles = torch.tensor([0, math.pi / 4, math.pi / 2, 3 * math.pi / 4])
sines = torch.sin(angles)
inverses = torch.asin(sines)
print("Sine and arcsine:")
print(f"angles\n{angles}")
print(f"sin(angles)\n{sines}")
print(f"asin(angles)\n{inverses}")

Sine and arcsine:
angles
tensor([0.0000, 0.7854, 1.5708, 2.3562])
sin(angles)
tensor([0.0000, 0.7071, 1.0000, 0.7071])
asin(angles)
tensor([0.0000, 0.7854, 1.5708, 0.7854])


In [49]:
print("Bitwise XOR:")
b = torch.tensor([1, 5, 11])
c = torch.tensor([2, 7, 10])
print(torch.bitwise_xor(b, c))

Bitwise XOR:
tensor([3, 2, 1])


In [50]:
print("Broadcasted element-wise equality comparison:")
d = torch.tensor([[1., 2.], [3., 4.]])
e = torch.ones(1, 2)
print(torch.eq(d, e))

Broadcasted element-wise equality comparison:
tensor([[ True, False],
        [False, False]])


In [51]:
print("Reduction operations:")
d = torch.tensor([[1., 2.], [3., 4.]])
e = torch.tensor([1, 2, 1, 2, 1, 2])
print(f"max(d)\n{torch.max(d)}")
print(f"max(d).item()\n{torch.max(d).item()}")
print(f"mean(d)\n{torch.mean(d)}")
print(f"std(d)\n{torch.std(d)}")
print(f"prod(d)\n{torch.prod(d)}")
print(f"unique(e)\n{torch.unique(e)}")

Reduction operations:
max(d)
4.0
max(d).item()
4.0
mean(d)
2.5
std(d)
1.29099440574646
prod(d)
24.0
unique(e)
tensor([1, 2])


In [52]:
print("Vectors and matrices:")
v1 = torch.tensor([1., 0., 0.])
v2 = torch.tensor([0., 1., 0.])
m1 = torch.rand(2, 2)
m2 = torch.tensor([[3., 0.], [0., 3.]])
m3 = torch.matmul(m1, m2)
print(f"cross(v2, v1)\n{torch.cross(v2, v1, dim=0)}")
print(f"matmul(m1, m2)\n{m3}")
print(f"svd(m3)\n{torch.svd(m3)}")

Vectors and matrices:
cross(v2, v1)
tensor([ 0.,  0., -1.])
matmul(m1, m2)
tensor([[1.4739, 2.6739],
        [0.4342, 1.5944]])
svd(m3)
torch.return_types.svd(
U=tensor([[-0.8826, -0.4702],
        [-0.4702,  0.8826]]),
S=tensor([3.4546, 0.3442]),
V=tensor([[-0.4356, -0.9001],
        [-0.9001,  0.4356]]))


## Tensor Metadata

In [53]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]

In [54]:
print(f"points.storage_offset(): {points.storage_offset()}")
print(f"points.size():           {points.size()}")
print(f"points.stride():         {points.stride()}")

points.storage_offset(): 0
points.size():           torch.Size([3, 2])
points.stride():         (2, 1)


In [55]:
print(f"second_point.storage_offset(): {second_point.storage_offset()}")
print(f"second_point.size():           {second_point.size()}")
print(f"second_point.stride():         {second_point.stride()}")

second_point.storage_offset(): 2
second_point.size():           torch.Size([2])
second_point.stride():         (1,)


In [56]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point[0] = 10.0
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [57]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1].clone()
second_point[0] = 10.0
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [58]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [59]:
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [60]:
id(points.untyped_storage()) == id(points_t.untyped_storage())

True

In [61]:
print(f"points.storage_offset():   {points.storage_offset()}")
print(f"points_t.storage_offset(): {points_t.storage_offset()}")

print(f"points.size():             {points.size()}")
print(f"points_t.size():           {points_t.size()}")

print(f"points.stride():           {points.stride()}")
print(f"points_t.stride():         {points_t.stride()}")

points.storage_offset():   0
points_t.storage_offset(): 0
points.size():             torch.Size([3, 2])
points_t.size():           torch.Size([2, 3])
points.stride():           (2, 1)
points_t.stride():         (1, 2)


In [62]:
points.is_contiguous()

True

In [63]:
points_t.is_contiguous()

False

In [64]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [65]:
points_t.storage()

/tmp/ipykernel_30900/3158410579.py:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  points_t.storage()


 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [66]:
points_t.stride()

(1, 2)

In [67]:
points_t_cont = points_t.contiguous()
points_t_cont

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [68]:
points_t_cont.storage()

 4.0
 5.0
 2.0
 1.0
 3.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [69]:
points_t_cont.stride()

(3, 1)

## NumPy Interoperability

In [70]:
points = torch.ones(3, 4)
points_np = points.numpy()
points_np

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

In [71]:
points = torch.from_numpy(points_np)
points

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

## Tensor Device

In [72]:
points_cpu = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]], device="cpu")
# points_gpu = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]], device="cuda")

In [73]:
points_cpu = points.to(device="cpu")
# points_gpu = points.to(device="cuda")
# points_gpu0 = points.to(device="cuda:0")

In [74]:
points_cpu = points.cpu()
# points_gpu = points.cuda()
# points_gpu0 = points.cuda(0)

## Serializing Tensors

In [75]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
torch.save(points, "data/ourpoints.t")

In [76]:
with open("data/ourpoints.t", "wb") as f:
    torch.save(points, f)

In [77]:
points = torch.load("data/ourpoints.t")

In [78]:
with open("data/ourpoints.t", "rb") as f:
    points = torch.load(f)

In [79]:
import h5py

In [80]:
f = h5py.File("data/ourpoints.hdf5", "w")
dataset = f.create_dataset(name="coords", data=points.numpy())
f.close()

In [81]:
f = h5py.File("data/ourpoints.hdf5", "r")
dataset = f["coords"]
last_points = dataset[-2:]
last_points

array([[5., 3.],
       [2., 1.]], dtype=float32)

In [82]:
last_points = torch.from_numpy(dataset[-2:])
f.close()

last_points

tensor([[5., 3.],
        [2., 1.]])

## Exercises

In [109]:
a = torch.tensor(list(range(9)))
print(f"a.size():           {a.size()}")
print(f"a.storage_offset(): {a.storage_offset()}")
print(f"a.stride():         {a.stride()}")

a.size():           torch.Size([9])
a.storage_offset(): 0
a.stride():         (1,)


In [110]:
b = a.view(3, 3)
id(a.untyped_storage()) == id(b.untyped_storage())

True

In [111]:
c = b[1:, 1:]
print(f"c.size():           {c.size()}")
print(f"c.storage_offset(): {c.storage_offset()}")
print(f"c.stride():         {c.stride()}")

c.size():           torch.Size([2, 2])
c.storage_offset(): 4
c.stride():         (3, 1)


In [112]:
sqrt_a = torch.sqrt(a)

In [113]:
a = a.float()

In [114]:
a.sqrt_()

tensor([0.0000, 1.0000, 1.4142, 1.7321, 2.0000, 2.2361, 2.4495, 2.6458, 2.8284])